In [ ]:
!pip install torchmetrics  # ==0.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 7.7 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00


In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.7 MB/s eta 0:00:00


In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.9 MB/s eta 0:00:00


In [ ]:
max_split_size_mb = 128

import torch
torch.cuda.empty_cache()


import gc
gc.collect()

0

In [ ]:
import os
import torch
from torch.utils.data import DataLoader
from torchmetrics.functional import accuracy, precision, recall, average_precision
from torchmetrics.classification import BinaryF1Score

from torchmetrics.functional import confusion_matrix, matthews_corrcoef
from transformers.integrations import WandbCallback
import pandas as pd

from transformers import BertConfig, BertForSequenceClassification, BertTokenizer
from transformers import RobertaConfig, RobertaTokenizer, RobertaForSequenceClassification

import csv
from transformers import DistilBertTokenizerFast
from transformers import DistilBertForSequenceClassification, TrainingArguments, Trainer
from transformers import DistilBertForSequenceClassification, AdamW

from transformers import (
    AdamW,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
)
import evaluate
import evaluate
import numpy as np
import wandb

from sklearn.model_selection import KFold

#os.environ["WANDB_PROJECT"]="rationale_project" # rationale_project
#os.environ["WB_ENTITY"]="rationale_project" # rationale_project


cols = ['commit number', 'commit ID', 'author name', 'committer name', 'message', 'URL', 'message_preprocessed', 'Decision', 'Rationale', 'Supporting Facts']

cols_to_index = {col:i for i,col in enumerate(cols)}



def decode_predictions(tokenizer, predictions):
    # labels = tokenizer.batch_decode(predictions.label_ids)
    labels = predictions.label_ids
    logits = predictions.predictions.argmax(axis=-1)
    prediction_text = logits
    return {"labels": labels, "predictions": prediction_text}




class WandbPredictionProgressCallback(WandbCallback):
    """Custom WandbCallback to log model predictions during training.

    This callback logs model predictions and labels to a wandb.Table at each
    logging step during training. It allows to visualize the
    model predictions as the training progresses.

    Attributes:
        trainer (Trainer): The Hugging Face Trainer instance.
        tokenizer (AutoTokenizer): The tokenizer associated with the model.
        sample_dataset (Dataset): A subset of the validation dataset
          for generating predictions.
        num_samples (int, optional): Number of samples to select from
          the validation dataset for generating predictions. Defaults to 100.
        freq (int, optional): Frequency of logging. Defaults to 2.
    """

    def __init__(self, trainer, tokenizer, val_dataset,
                 num_samples=100, freq=1):
        """Initializes the WandbPredictionProgressCallback instance.

        Args:
            trainer (Trainer): The Hugging Face Trainer instance.
            tokenizer (AutoTokenizer): The tokenizer associated
              with the model.
            val_dataset (Dataset): The validation dataset.
            num_samples (int, optional): Number of samples to select from
              the validation dataset for generating predictions.
              Defaults to 100.
            freq (int, optional): Frequency of logging. Defaults to 2.
        """
        super().__init__()
        self.trainer = trainer
        self.tokenizer = tokenizer
        self.val_dataset = val_dataset
        self.freq = freq

    def on_evaluate(self, args, state, control, **kwargs):
        super().on_evaluate(args, state, control, **kwargs)
        # control the frequency of logging by logging the predictions
        # every `freq` epochs
        if state.epoch % self.freq == 0:
            # generate predictions
            predictions = self.trainer.predict(self.val_dataset)
            # decode predictions and labels
            predictions = decode_predictions(self.tokenizer, predictions)

            y = torch.tensor(predictions['labels'])
            preds = torch.tensor(predictions['predictions'])
            stage = "" #"val"
            f1 = BinaryF1Score()
            metrics_table = {
                f"{stage}accuracy": accuracy(preds, y, task ='binary'),
                f'{stage}precision': precision(preds, y, average='none',task ='binary', num_classes=2 ),
                f'{stage}recall': recall(preds, y, average='none', num_classes=2, task ='binary'),
                #f'{stage}f1_score_macro': f1(preds, y, average='macro', num_classes=2),
                f'{stage}f1_score': f1(preds, y),
                # 'avg_precision' : average_precision(preds, y),
                #f'{stage}mcc': matthews_corrcoef(preds, y, num_classes=2),
            }
            print(f"Epoch {int(state.epoch)}:")
            for key, val in metrics_table.items():
                self._wandb.log({key: val})
                print(f"{key}\t{val:.4f}")

            # Append metrics to lists
            accuracy_list.append(metrics_table['accuracy'])
            precision_list.append(metrics_table['precision'])
            recall_list.append(metrics_table['recall'])
            f1_list.append(metrics_table['f1_score'])

            # self._wandb.log({"metrics_table": metrics_table})

            # # add predictions to a wandb.Table
            # # predictions_df = pd.DataFrame(predictions)
            # # predictions_df["epoch"] = state.epoch
            # # predictions_df["message"] = [x for x in DataLoader(self.val_dataset)]
            # records_table = self._wandb.Table(dataframe=predictions_df)
            # # log the table to wandb
            # self._wandb.log({"sample_predictions": records_table})


def read_data(file, category="Rationale"):
    data = []
    labels = []
    choices = ['Decision', 'Rationale']
    if category not in choices:
        raise ValueError(f"Category must be in {choices}")
    stats = {"yes":0, "no":0, "total": 0}
    with open(file, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter = ';')
        next(reader)
        for row in reader:
            data.append(row[cols_to_index['message_preprocessed']])
            if row[cols_to_index[category]] == "yes":
                label = 1
                stats['yes']+=1
            else:
                label = 0
                stats['no'] += 1
            stats['total'] += 1
            labels.append(label)
    print(f"Loaded {category} data: {stats}")
    return data, labels


class RationaleDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


#def compute_metrics(eval_preds):
 #   metric = evaluate.load("glue", "mrpc")
  #  logits, labels = eval_preds
   # predictions = np.argmax(logits, axis=-1)
    #return metric.compute(predictions=predictions, references=labels)


if __name__ == '__main__':

    #models = ['distilbert-base-uncased']
    models = ['distilbert-base-uncased' ,'albert-base-v2', 'bert-base-cased', 'roberta-base', 'albert-large-v2', 'bert-large-cased', 'roberta-large']

    tasks = ['Decision', 'Rationale']


    AVAIL_GPUS = min(1, torch.cuda.device_count())
    NUM_WORKERS = 0  # int(os.cpu_count() / 2)
    TOKENIZERS_PARALLELISM = True
    print(f"AVAIL_GPUS: {AVAIL_GPUS}")

    torch.manual_seed(42)

    for selected_model in models:
      for selected_task in tasks:

        accuracy_list, precision_list, recall_list, f1_list = [], [], [], []

        params=dict(epochs=5,model_name=selected_model,task=selected_task) # we chnage the number of epochs

        model_version = f"{params['model_name']}-{params['task']}-e-{params['epochs']}--CV"
        print("*-*-"*200)
        print("*-*-"*200)
        print(" Start ", model_version)
        print("*-*-"*200)
        print("*-*-"*200)

        run = wandb.init(
            project="rationale_project",
            entity="rationale_project",
            group=selected_task,  # or args.eval_data
            tags=[selected_task, selected_model],
            name=model_version,
            config=params,
            reinit = True,
            settings=wandb.Settings(start_method="fork"))


        texts, labels = read_data("dataset_3_labels_merged.csv", params['task'])

        kfold = KFold(n_splits=10, shuffle=True, random_state=1)

        for fold, (train_indices, test_indices) in enumerate(kfold.split(texts, labels)):

                print(f"Training on fold {fold + 1}...")
                train_texts, test_texts = [texts[i] for i in train_indices], [texts[i] for i in test_indices]
                train_labels, test_labels = [labels[i] for i in train_indices], [labels[i] for i in test_indices]

                 # train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)
                stats_train = {'yes':train_labels.count(1),'no':train_labels.count(0),'total':len(train_labels)}
                stats_test = {'yes':test_labels.count(1),'no':test_labels.count(0),'total': len(test_labels)}
                print("training data:", stats_train)
                print("test data:", stats_test)
                # tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
                tokenizer = AutoTokenizer.from_pretrained(params['model_name'], use_fast=True)

                train_encodings = tokenizer(train_texts, truncation=True, padding=True)
                test_encodings = tokenizer(test_texts, truncation=True, padding=True)

                train_dataset = RationaleDataset(train_encodings, train_labels)
                test_dataset = RationaleDataset(test_encodings, test_labels)

                config = AutoConfig.from_pretrained(params['model_name'], num_labels=2)
                model = AutoModelForSequenceClassification.from_pretrained(params['model_name'], config=config)

                training_args = TrainingArguments(
                    output_dir='./results',
                    num_train_epochs=params['epochs'],
                    per_device_train_batch_size=16,
                    per_device_eval_batch_size=64,
                    warmup_steps=500,
                    weight_decay=0.01,
                    logging_dir='./logs',
                    logging_steps=10,
                    evaluation_strategy="epoch"
                )

                trainer = Trainer(
                    model=model,
                    args=training_args,
                    tokenizer=tokenizer,
                    train_dataset=train_dataset,
                    eval_dataset=test_dataset
                )

                progress_callback = WandbPredictionProgressCallback(
                    trainer=trainer,
                    tokenizer=tokenizer,
                    val_dataset=test_dataset,
                    num_samples=10,
                    freq=5, # every three/five epochs, so every 1 fold
                )

                trainer.add_callback(progress_callback)
                trainer.train()


        # Calculate average metrics across all folds
        avg_accuracy = np.mean(accuracy_list)
        avg_precision = np.mean(precision_list)
        avg_recall = np.mean(recall_list)
        avg_f1 = np.mean(f1_list)

        print(f"LEN ACCURACY LIST: {len(accuracy_list)}" ) # should be 10

        print("\nAverage Metrics Across Folds:", model_version)
        print(f"Average Accuracy: {avg_accuracy}")
        print(f"Average Precision: {avg_precision}")
        print(f"Average Recall: {avg_recall}")
        print(f"Average F1 Score: {avg_f1}")




AVAIL_GPUS: 1
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


...

Epoch,Training Loss,Validation Loss
1,0.666600,0.631728
2,0.595800,0.457765
3,0.419500,0.457136


Epoch 3:
accuracy	0.8080
precision	0.7943
recall	0.8889
f1_score	0.8390
Training on fold 2...
training data: {'yes': 1129, 'no': 881, 'total': 2010}
test data: {'yes': 123, 'no': 101, 'total': 224}


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 